<a href="https://colab.research.google.com/github/2ovisa/AH2179/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stop ahead prediction
- intressant för resenärer att veta hur stor förseningen blir vid sitt stop
- intressant för resenärer att veta hur stor förseningen är vid sin påstigande station

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

#-------------------------------------------------data preprocessing------------------------------------------------------------------

url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/ProjectAssignmentData/Dataset-PT.csv'
df = pd.read_csv(url, header = 1) #skip the first row and use the second as the header
df.head(10)
#df.shape
#print(df.columns.tolist())

df.iloc[:, 0:14] = df.iloc[:, 0:14].astype(float)
df.iloc[:, 14:18] = df.iloc[:, 14:18].astype(str)
df.iloc[:, 18:31] = df.iloc[:, 18:31].astype(int)
df = df.iloc[:1000]




/tmp/ipython-input-1390479708.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         20220108.0
1         20220108.0
2         20220108.0
3         20220108.0
4         20220108.0
             ...    
545098    20220630.0
545099    20220630.0
545100    20220630.0
545101    20220630.0
545102    20220630.0
Name: Calendar_date, Length: 545103, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0:14] = df.iloc[:, 0:14].astype(float)
/tmp/ipython-input-1390479708.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         4.0
1         4.0
2         4.0
3         4.0
4         4.0
         ... 
545098    4.0
545099    4.0
545100    4.0
545101    4.0
545102    4.0
Name: route_id, Length: 545103, dtype: float64' has dtype incompatible with int64, please explicitly cast to a co

###Kolumner att droppa:
- Identifiers: Calender_date, route_id, bus_id (innehåller inte nödvändig information
- They who leak future knowledge: Arrival_delay (should be target)
- Dummies: To avoid multi collineraty one of the examples should be dropped. factor(temperature)Normal, (weather)normal, weekend, off-peak. *these become the baseline?*


In [ ]:
target = "arrival_delay"
#Create multi-horizon shifted targets
# Assuming stop_sequence roughly corresponds to future steps
#this is a stop-ahead prediction
df["delay_t+1_stop"]  = df.groupby("bus_id")[target].shift(-1)   # next stop
df["delay_t+3_stop"] = df.groupby("bus_id")[target].shift(-3)   # ~3 stops ahead
df["delay_t+6_stop"] = df.groupby("bus_id")[target].shift(-6)
df["delay_t+12_stop"] = df.groupby("bus_id")[target].shift(-12)

#Drop rows with missing target values (at trip ends)
df = df.dropna(subset=["delay_t+1_stop", "delay_t+3_stop", "delay_t+6_stop", "delay_t+12_stop"])





*Why is arrival_delay used and not stop sequence?*
- *stop sequence is just an index telling which stop number in the trip. It only contains ordering, not delay information*
- *the code says "for stop i, the target is the arrival delay at stop i+1*

*this assumes each stop is ~5 minutes apart. If travel times vary a lot, then “t+5 min” might not really equal “next stop”*

In [ ]:
bus_sample = df[df["bus_id"] == df["bus_id"].iloc[0]].sort_values("stop_sequence")

print(bus_sample[["stop_sequence", "arrival_delay",
                  "delay_t+1_stop", "delay_t+3_stop", "delay_t+6_stop", "delay_t+12_stop"]].head(20))

In [ ]:
avg_travel = df["scheduled_travel_time"].mean()
print("Average travel time between stops (seconds):", avg_travel)

In [8]:
# Drop specific columns (Arrival_time, Stop_id, Bus_id, Line_id) from the DataFrame.
df_model = df.drop(['Calendar_date', 'route_id'], axis=1)
#df = pd.get_dummies(df.drop(['factor(weather)Normal', 'factor(temperature)Normal', 'factor(day_of_week)weekend', 'factor(time_of_day)Off-peak'], axis = 1)) #Dummies

categorical_cols = [
    'weather', 'temperature', 'day_of_week', 'time_of_day',
    'factor(weather)Light_Rain', 'factor(weather)Light_Snow',
    'factor(weather)Rain', 'factor(weather)Snow',
    'factor(temperature)Cold', 'factor(temperature)Extra_cold',
    'factor(day_of_week)weekday',
    'factor(time_of_day)Afternoon_peak', 'factor(time_of_day)Morning_peak'
]
df_model = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

target = "arrival_delay"

#create multi horizon shifted target using stop sequence for proper order
df_model = df_model.sort_values(['bus_id', 'stop_sequence'])
df_model["delay_1_stop"]  = df_model.groupby("bus_id")[target].shift(-1)   # next stop
df_model["delay_3_stop"] = df_model.groupby("bus_id")[target].shift(-3)   # ~3 stops ahead
df_model["delay_6_stop"] = df_model.groupby("bus_id")[target].shift(-6)   # ~6 stops ahead
df_model["delay_12_stop"] = df_model.groupby("bus_id")[target].shift(-12)  # ~12 stops ahead

#drop rows that hare missing future delays, like the end of trips
df_model = df_model.dropna(subset=["delay_1_stop", "delay_3_stop", "delay_6_stop", "delay_12_stop"])


#Split into X (features) and y (multi-output)
X = df_model.drop(['arrival_delay', "delay_1_stop", "delay_3_stop", "delay_6_stop", "delay_12_stop"], axis =1)
y = df_model[["delay_1_stop", "delay_3_stop", "delay_6_stop", "delay_12_stop"]]

#X = X.astype(int)

# splite the train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("X shape:", X.shape)
print("y shape:", y.shape)
print(y.head())

X shape: (783, 19)
y shape: (783, 4)
     delay_1_stop  delay_3_stop  delay_6_stop  delay_12_stop
0           106.0         185.0         186.0          242.0
162           3.0         116.0          99.0          183.0
297         185.0          37.0          53.0           90.0
1           116.0         186.0         202.0          260.0
163          37.0          99.0         118.0          209.0


- *if stop_sequence is a freature, the model might cheat by just learning that higher stop numbers mean later in the trip*

##Multi-output with KNN

In [9]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error

model = KNeighborsRegressor()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)


#evaluate
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
for col, score in zip(y.columns, mae):
    print(f"MAE for {col}: {score:.2f}")



MAE for delay_1_stop: 63.44
MAE for delay_3_stop: 50.24
MAE for delay_6_stop: 42.76
MAE for delay_12_stop: 51.29


MAE = the average absolute difference between the models predictions and the actual delay

- MAE for delay_t+1_stop: 34.00
- MAE for delay_t+3_stop: 46.54
- MAE for delay_t+6_stop: 54.66
- MAE for delay_t+12_stop: 56.01

ANDra gången: MAE for delay_1_stop: 63.44
MAE for delay_3_stop: 50.24
MAE for delay_6_stop: 42.76
MAE for delay_12_stop: 51.29


This means that on average, the prediction for the next stop is off by 34 seconds.

Interpretations:
- Errors grow as the horizon length increases, which is expected since uncertainty accumulates further into the future


#Visualization

In [ ]:
import matplotlib.pyplot as plt

# Define horizons
horizons = ["delay_t+1_stop", "delay_t+3_stop", "delay_t+6_stop", "delay_t+12_stop"]
n_horizons = len(horizons)

plt.figure(figsize=(15, 10))

for i, horizon in enumerate(horizons):
    plt.subplot(2, 2, i+1)
    plt.scatter(y_test[horizon], y_pred[:, i], alpha=0.5)
    plt.plot([y_test[horizon].min(), y_test[horizon].max()],
             [y_test[horizon].min(), y_test[horizon].max()],
             color='red', linestyle='--')  # perfect prediction line
    plt.xlabel("Actual Delay (s)")
    plt.ylabel("Predicted Delay (s)")
    plt.title(f"Predicted vs Actual: {horizon}")

plt.tight_layout()
plt.show()


- *points above the line = overestimations, below = underestimations*

In [ ]:
import matplotlib.pyplot as plt

# Select a single bus trip (bus_id and date) to visualize
trip_id = 41344  # Example bus_id, adjust if needed
trip_data = df[df['bus_id'] == trip_id].sort_values('stop_sequence')

# Assuming y_pred is a NumPy array of predictions for this trip
# Make sure predictions correspond to this trip
predictions = y_pred[:len(trip_data), :]  # adjust indexing if needed

# Horizons
horizons = ["delay_t+5", "delay_t+15", "delay_t+30", "delay_t+60"]

plt.figure(figsize=(12, 6))

for i, horizon in enumerate(horizons):
    plt.plot(trip_data['stop_sequence'], trip_data[horizon], marker='o', label=f'Actual {horizon}')
    plt.plot(trip_data['stop_sequence'], predictions[:, i], marker='x', linestyle='--', label=f'Predicted {horizon}')

plt.xlabel("Stop Sequence")
plt.ylabel("Arrival Delay (s)")
plt.title(f"Delay Propagation Along Stops for Bus {trip_id}")
plt.legend()
plt.grid(True)
plt.show()
